In [ ]:
from selenium import webdriver
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd

#Connect to C2Q
driver = webdriver.Chrome("/Users/sal/Downloads/chromedriver")
driver.get("http://www.ohdsi.org/web/criteria2query/")

#Load clinical trial
nctid = driver.find_element_by_id('nctid')
fetchct = driver.find_element_by_id('fetchct')
nctid.send_keys("NCT03880240")
ActionChains(driver).click(fetchct).perform()

#JSON Query
#auto = driver.find_element_by_id('auto')
#ActionChains(driver).click(auto).perform()

#Parse the criteria
parse = driver.find_element_by_id('start')
ActionChains(driver).click(parse).perform()

#Extract JSON text
wait = WebDriverWait(driver, 20)
wait.until(EC.presence_of_element_located((By.ID, 'downloadfile')))
download = driver.find_element_by_id('downloadfile')
ActionChains(driver).click(download).perform()

In [117]:
import json
import pandas as pd

with open("criteria2query.json", "r") as read_file:
    data = json.load(read_file)

trial = pd.DataFrame.from_dict(data)    
print (data)

{'eligibility criteria': {'exclusion_criteria': [{'include': False, 'pattern': 0, 'pid': 0, 'sents': [{'display': ' Current history of <mark data-entity="qualifier">poorly</mark><mark data-entity="condition">controlled migraines</mark>including <mark data-entity="drug">chronic medication</mark>for <mark data-entity="condition">migraine</mark>prevention ', 'ignore': False, 'include': False, 'logic_groups': [], 'relations': [], 'terms': [{'categorey': 'Condition', 'end_index': 47, 'neg': False, 'start_index': 27, 'termId': 1, 'text': 'controlled migraines'}, {'categorey': 'Drug', 'end_index': 76, 'neg': False, 'start_index': 58, 'termId': 2, 'text': 'chronic medication'}, {'categorey': 'Condition', 'end_index': 89, 'neg': False, 'start_index': 81, 'termId': 3, 'text': 'migraine'}], 'text': ' Current history of poorly controlled migraines including chronic medication for migraine prevention '}]}, {'include': False, 'pattern': 0, 'pid': 0, 'sents': [{'display': ' <mark data-entity="tempora

trial[0,0] is the exclusion criteria
trial[1,0] is the inclusion criteria

In [118]:
exclusion = trial.iat[0,0]
inclusion = trial.iat[1,0]
print (exclusion[0].get('sents')[0].get('terms')[0])

{'categorey': 'Condition', 'end_index': 47, 'neg': False, 'start_index': 27, 'termId': 1, 'text': 'controlled migraines'}


In [119]:
m = []
for e in exclusion:
    phrase = e.get('sents')[0].get('text')
    q = []
    for t in e.get('sents')[0].get('terms'):
        entity = t.get('text')
        domain = t.get('categorey')
        q.append([entity, domain])
    m.append({phrase:q})

for i in inclusion:
    phrase = i.get('sents')[0].get('text')
    q = []
    for t in i.get('sents')[0].get('terms'):
        entity = t.get('text')
        domain = t.get('categorey')
        q.append([entity, domain])
    m.append({phrase:q})

In [120]:
m

[{' Current history of poorly controlled migraines including chronic medication for migraine prevention ': [['controlled migraines',
    'Condition'],
   ['chronic medication', 'Drug'],
   ['migraine', 'Condition']]},
 {' Current or past history of any neurological disorder other than dementia , such as epilepsy , stroke ( cortical stroke ) , progressive neurologic disease ( e.g. multiple sclerosis ) or intracranial brain lesions ; and history of previous neurosurgery or head trauma that resulted in residual neurologic impairment . ': [['neurological disorder',
    'Condition'],
   ['dementia', 'Condition'],
   ['epilepsy', 'Condition'],
   ['stroke', 'Condition'],
   ['cortical stroke', 'Condition'],
   ['progressive neurologic disease', 'Condition'],
   ['multiple sclerosis', 'Condition'],
   ['intracranial brain lesions', 'Condition'],
   ['neurosurgery', 'Procedure'],
   ['head trauma', 'Condition'],
   ['residual neurologic impairment', 'Condition'],
   ['Current', 'Temporal'],
  